## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
display(dbutils.fs.ls("/FileStore/tables"))

path,name,size
dbfs:/FileStore/tables/Db.db,Db.db,22425600
dbfs:/FileStore/tables/ReadMe-1.txt,ReadMe-1.txt,407
dbfs:/FileStore/tables/ReadMe.txt,ReadMe.txt,407
dbfs:/FileStore/tables/ipl/,ipl/,0
dbfs:/FileStore/tables/ipl_ball_by_ball.csv,ipl_ball_by_ball.csv,20716091
dbfs:/FileStore/tables/ipl_dataset_2-1.zip,ipl_dataset_2-1.zip,3085489
dbfs:/FileStore/tables/ipl_dataset_2.zip,ipl_dataset_2.zip,3085489
dbfs:/FileStore/tables/ipl_matches.csv,ipl_matches.csv,124625
dbfs:/FileStore/tables/ipl_venue.csv,ipl_venue.csv,1410


In [0]:
import os

In [0]:
# File location and type
folder_loc = "/FileStore/tables"
# The applied options are for CSV files. For other file types, these will be ignored.
df_match = spark.read.csv(os.path.join(folder_loc,'ipl_matches.csv'),header=True,inferSchema=True)
df_venue = spark.read.csv(os.path.join(folder_loc,'ipl_venue.csv'),header=True,inferSchema=True)
df_ball = spark.read.csv(os.path.join(folder_loc,'ipl_ball_by_ball.csv'),header=True,inferSchema=True)
display(df_match.show(5),df_venue.show(5),df_ball.show(5))

+--------+----------+---------------+--------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+----------+------+---------+--------------+
|match_id|      date|player_of_match|venue_id|neutral_venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|eliminator|method|  umpire1|       umpire2|
+--------+----------+---------------+--------+-------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+----------+------+---------+--------------+
|  335982|2008-04-18|    BB McCullum|      35|            0|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knight Ri...|   runs|          140|         N|    NA|Asad Rauf|   RE Koertzen|
|  335983|2008-04-19|     MEK Hussey|      31|            0|     Kings XI Punjab| Chennai Super Kings| C

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL").getOrCreate()

All the Queries have been edited as the title of the cell.

In [0]:
df_match.filter(df_match.eliminator == "Y").groupby(['venue_id']).count().\
         join(df_venue,on='venue_id').sort('count',ascending = False).select('venue').show(3,truncate = False)

+-----------------------------------+
|venue                              |
+-----------------------------------+
|Dubai International Cricket Stadium|
|Sheikh Zayed Stadium               |
|Feroz Shah Kotla                   |
+-----------------------------------+
only showing top 3 rows



In [0]:
df_ball.filter(df_ball.dismissal_kind == 'caught' ).groupby(['fielder']).count().sort('count',ascending = False).select('count').show(1)

+-----+
|count|
+-----+
|  118|
+-----+
only showing top 1 row



In [0]:
temp = df_ball.join(df_match.filter(df_match.method == 'D/L').select('match_id'),on = 'match_id')
temp.filter((temp.is_wicket == 1) & (temp.dismissal_kind != 'stumped') & (temp.extras_type != 'noballs')).groupby('bowler').count()\
                      .sort('count',ascending = False).show(1)              

+--------+-----+
|  bowler|count|
+--------+-----+
|AB Dinda|    8|
+--------+-----+
only showing top 1 row



In [0]:
total_run = df_ball.filter(df_ball.overs>6).groupby('batsman').sum('batsman_runs').withColumnRenamed("sum(batsman_runs)","runs")
total_balls = df_ball.filter((df_ball.overs>6)&(df_ball.extras_type != 'noballs')).groupby('batsman')\
              .count().withColumnRenamed("count","ball")
temp = total_run.join(total_balls,on='batsman')
temp.withColumn('SR',temp.runs*100/temp.ball).sort('SR',ascending=False).show(1)

+----------+----+----+-----+
|   batsman|runs|ball|   SR|
+----------+----+----+-----+
|B Stanlake|   5|   2|250.0|
+----------+----+----+-----+
only showing top 1 row



In [0]:
# Highest Strike Rate by batsmen metric is certainly not the best way as a batsman might have faced only 2-3 balls and made some boundary which will make the Strike Rate too high , in such cases the situation is not represented properly as it is better to have a batsman with a bit lower strike rate but have had more experience .
# We will only consider those batsman that have played more than 6 balls atleast .
total_run = df_ball.filter(df_ball.overs>6).groupby('batsman').sum('batsman_runs').withColumnRenamed("sum(batsman_runs)","runs")
total_balls = df_ball.filter((df_ball.overs>6)&(df_ball.extras_type != 'noballs')).groupby('batsman')\
              .count().withColumnRenamed("count","ball")
total_balls = total_balls.filter(total_balls.ball>6)
temp = total_run.join(total_balls,on='batsman')
temp.withColumn('SR',temp.runs*100/temp.ball).sort('SR',ascending=False).show(1)

+------------+----+----+------------------+
|     batsman|runs|ball|                SR|
+------------+----+----+------------------+
|Kamran Akmal|  77|  36|213.88888888888889|
+------------+----+----+------------------+
only showing top 1 row



In [0]:
temp = df_ball.groupby('match_id').sum('extra_runs').withColumnRenamed("sum(extra_runs)","extras")
temp.join(df_match.select(['match_id','venue_id']),temp.match_id == df_match.match_id).groupby('venue_id').sum('extras').\
    withColumnRenamed("sum(extras)","extras").join(df_venue,on = 'venue_id').sort('extras',ascending=False).\
    select('venue','city','extras').show(1,truncate=False)

+---------------------+---------+------+
|venue                |city     |extras|
+---------------------+---------+------+
|M.Chinnaswamy Stadium|Bengaluru|1355  |
+---------------------+---------+------+
only showing top 1 row



In [0]:
temp = df_match.filter(df_match.neutral_venue == 1).groupby('player_of_match').count().withColumnRenamed("count","counter")
temp.filter(temp.counter == temp.agg({"counter": "max"}).collect()[0][0]).show()

+---------------+-------+
|player_of_match|counter|
+---------------+-------+
|      YK Pathan|      3|
|     GJ Maxwell|      3|
|      JH Kallis|      3|
|      JP Duminy|      3|
+---------------+-------+



In [0]:
total_run = df_ball.groupby('batsman').sum('batsman_runs').withColumnRenamed("sum(batsman_runs)","runs")
total_outs = df_ball.filter(df_ball.player_dismissed != 'NA').groupby('player_dismissed').count().\
              withColumnRenamed("count","outs").withColumnRenamed("player_dismissed","batsman")
temp = total_run.join(total_outs,on='batsman')
temp.withColumn('AVG',temp.runs/temp.outs).sort('AVG',ascending=False).select('batsman').show(10)

+-------------+
|      batsman|
+-------------+
|   MN van Wyk|
|   RD Gaikwad|
|     AC Voges|
|     KL Rahul|
|      HM Amla|
|Iqbal Abdulla|
|    DA Warner|
|  JM Bairstow|
|     CH Gayle|
|     MS Dhoni|
+-------------+
only showing top 10 rows



In [0]:
ump_1 = df_match.groupby('umpire1').count().withColumnRenamed("count","umpire_1")
ump_2 = df_match.groupby('umpire2').count().withColumnRenamed("count","umpire_2")
ump_total = ump_1.join(ump_2,on = (ump_1.umpire1 == ump_2.umpire2))
ump_total.withColumn('Total_officiates',ump_total.umpire_1+ump_total.umpire_2).sort('Total_officiates',ascending=False)\
         .select('umpire1').show(1)

+-------+
|umpire1|
+-------+
| S Ravi|
+-------+
only showing top 1 row



In [0]:
temp = df_ball.filter(df_ball.batsman == 'V Kohli').groupby('match_id').sum('batsman_runs').sort('sum(batsman_runs)',ascending=False)\
       .take(1)[0][0]
temp = df_match.filter(df_match.match_id == temp).select('venue_id').take(1)[0][0]
df_venue.filter(df_venue.venue_id == temp).select(['venue','city']).show(truncate=False)

+---------------------+---------+
|venue                |city     |
+---------------------+---------+
|M.Chinnaswamy Stadium|Bengaluru|
+---------------------+---------+



In [0]:
# Creative Case study: Please analyze how winning/losing tosses can impact a match and it's result?
from pyspark.sql.functions import when
df_match = spark.read.csv(os.path.join(folder_loc,'ipl_matches.csv'),header=True,inferSchema=True)
df_match = df_match.withColumn('Wins toss and match',df_match.winner == df_match.toss_winner)
df_match = df_match.withColumn('Toss & Match',when(df_match['Wins toss and match'],'Win Both')\
                               .when(df_match['Wins toss and match'] == False ,'Lost Toss or Match'))
df_match = df_match.join(df_venue,on = "venue_id")
display(df_match.select('Toss & Match'))

Toss & Match
Lost Toss or Match
Win Both
Lost Toss or Match
Lost Toss or Match
Lost Toss or Match
Lost Toss or Match
Lost Toss or Match
Lost Toss or Match
Win Both
Lost Toss or Match


In [0]:
display(df_match)

venue_id,match_id,date,player_of_match,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,Wins toss and match,Toss & Match,venue,city
35,335982,2008-04-18,BB McCullum,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,N,NA,Asad Rauf,RE Koertzen,false,Lost Toss or Match,M.Chinnaswamy Stadium,Bengaluru
31,335983,2008-04-19,MEK Hussey,0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33,N,NA,MR Benson,SL Shastri,true,Win Both,"Punjab Cricket Association Stadium, Mohali",Chandigarh
27,335984,2008-04-19,MF Maharoof,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9,N,NA,Aleem Dar,GA Pratapkumar,false,Lost Toss or Match,Feroz Shah Kotla,Delhi
14,335985,2008-04-20,MV Boucher,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5,N,NA,SJ Davis,DJ Harper,false,Lost Toss or Match,Wankhede Stadium,Mumbai
15,335986,2008-04-20,DJ Hussey,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5,N,NA,BF Bowden,K Hariharan,false,Lost Toss or Match,Eden Gardens,Kolkata
20,335987,2008-04-21,SR Watson,0,Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,bat,Rajasthan Royals,wickets,6,N,NA,Aleem Dar,RB Tiffin,false,Lost Toss or Match,Sawai Mansingh Stadium,Jaipur
22,335988,2008-04-22,V Sehwag,0,Deccan Chargers,Delhi Daredevils,Deccan Chargers,bat,Delhi Daredevils,wickets,9,N,NA,IL Howell,AM Saheba,false,Lost Toss or Match,"Rajiv Gandhi International Stadium, Uppal",Hyderabad
29,335989,2008-04-23,ML Hayden,0,Chennai Super Kings,Mumbai Indians,Mumbai Indians,field,Chennai Super Kings,runs,6,N,NA,DJ Harper,GA Pratapkumar,false,Lost Toss or Match,"MA Chidambaram Stadium, Chepauk",Chennai
22,335990,2008-04-24,YK Pathan,0,Deccan Chargers,Rajasthan Royals,Rajasthan Royals,field,Rajasthan Royals,wickets,3,N,NA,Asad Rauf,MR Benson,true,Win Both,"Rajiv Gandhi International Stadium, Uppal",Hyderabad
31,335991,2008-04-25,KC Sangakkara,0,Kings XI Punjab,Mumbai Indians,Mumbai Indians,field,Kings XI Punjab,runs,66,N,NA,Aleem Dar,AM Saheba,false,Lost Toss or Match,"Punjab Cricket Association Stadium, Mohali",Chandigarh


In [0]:
display(df_match)

venue_id,match_id,date,player_of_match,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,Wins toss and match,Toss & Match,venue,city
35,335982,2008-04-18,BB McCullum,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,N,NA,Asad Rauf,RE Koertzen,false,Lost Toss or Match,M.Chinnaswamy Stadium,Bengaluru
31,335983,2008-04-19,MEK Hussey,0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33,N,NA,MR Benson,SL Shastri,true,Win Both,"Punjab Cricket Association Stadium, Mohali",Chandigarh
27,335984,2008-04-19,MF Maharoof,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9,N,NA,Aleem Dar,GA Pratapkumar,false,Lost Toss or Match,Feroz Shah Kotla,Delhi
14,335985,2008-04-20,MV Boucher,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5,N,NA,SJ Davis,DJ Harper,false,Lost Toss or Match,Wankhede Stadium,Mumbai
15,335986,2008-04-20,DJ Hussey,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5,N,NA,BF Bowden,K Hariharan,false,Lost Toss or Match,Eden Gardens,Kolkata
20,335987,2008-04-21,SR Watson,0,Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,bat,Rajasthan Royals,wickets,6,N,NA,Aleem Dar,RB Tiffin,false,Lost Toss or Match,Sawai Mansingh Stadium,Jaipur
22,335988,2008-04-22,V Sehwag,0,Deccan Chargers,Delhi Daredevils,Deccan Chargers,bat,Delhi Daredevils,wickets,9,N,NA,IL Howell,AM Saheba,false,Lost Toss or Match,"Rajiv Gandhi International Stadium, Uppal",Hyderabad
29,335989,2008-04-23,ML Hayden,0,Chennai Super Kings,Mumbai Indians,Mumbai Indians,field,Chennai Super Kings,runs,6,N,NA,DJ Harper,GA Pratapkumar,false,Lost Toss or Match,"MA Chidambaram Stadium, Chepauk",Chennai
22,335990,2008-04-24,YK Pathan,0,Deccan Chargers,Rajasthan Royals,Rajasthan Royals,field,Rajasthan Royals,wickets,3,N,NA,Asad Rauf,MR Benson,true,Win Both,"Rajiv Gandhi International Stadium, Uppal",Hyderabad
31,335991,2008-04-25,KC Sangakkara,0,Kings XI Punjab,Mumbai Indians,Mumbai Indians,field,Kings XI Punjab,runs,66,N,NA,Aleem Dar,AM Saheba,false,Lost Toss or Match,"Punjab Cricket Association Stadium, Mohali",Chandigarh


In [0]:
display(df_match)

venue_id,match_id,date,player_of_match,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,Wins toss and match,Toss & Match,venue,city
35,335982,2008-04-18,BB McCullum,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,N,NA,Asad Rauf,RE Koertzen,false,Lost Toss or Match,M.Chinnaswamy Stadium,Bengaluru
31,335983,2008-04-19,MEK Hussey,0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33,N,NA,MR Benson,SL Shastri,true,Win Both,"Punjab Cricket Association Stadium, Mohali",Chandigarh
27,335984,2008-04-19,MF Maharoof,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9,N,NA,Aleem Dar,GA Pratapkumar,false,Lost Toss or Match,Feroz Shah Kotla,Delhi
14,335985,2008-04-20,MV Boucher,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5,N,NA,SJ Davis,DJ Harper,false,Lost Toss or Match,Wankhede Stadium,Mumbai
15,335986,2008-04-20,DJ Hussey,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5,N,NA,BF Bowden,K Hariharan,false,Lost Toss or Match,Eden Gardens,Kolkata
20,335987,2008-04-21,SR Watson,0,Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,bat,Rajasthan Royals,wickets,6,N,NA,Aleem Dar,RB Tiffin,false,Lost Toss or Match,Sawai Mansingh Stadium,Jaipur
22,335988,2008-04-22,V Sehwag,0,Deccan Chargers,Delhi Daredevils,Deccan Chargers,bat,Delhi Daredevils,wickets,9,N,NA,IL Howell,AM Saheba,false,Lost Toss or Match,"Rajiv Gandhi International Stadium, Uppal",Hyderabad
29,335989,2008-04-23,ML Hayden,0,Chennai Super Kings,Mumbai Indians,Mumbai Indians,field,Chennai Super Kings,runs,6,N,NA,DJ Harper,GA Pratapkumar,false,Lost Toss or Match,"MA Chidambaram Stadium, Chepauk",Chennai
22,335990,2008-04-24,YK Pathan,0,Deccan Chargers,Rajasthan Royals,Rajasthan Royals,field,Rajasthan Royals,wickets,3,N,NA,Asad Rauf,MR Benson,true,Win Both,"Rajiv Gandhi International Stadium, Uppal",Hyderabad
31,335991,2008-04-25,KC Sangakkara,0,Kings XI Punjab,Mumbai Indians,Mumbai Indians,field,Kings XI Punjab,runs,66,N,NA,Aleem Dar,AM Saheba,false,Lost Toss or Match,"Punjab Cricket Association Stadium, Mohali",Chandigarh


In [0]:
display(df_match)

venue_id,match_id,date,player_of_match,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,Wins toss and match,Toss & Match,venue,city
35,335982,2008-04-18,BB McCullum,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,N,NA,Asad Rauf,RE Koertzen,false,Lost Toss or Match,M.Chinnaswamy Stadium,Bengaluru
31,335983,2008-04-19,MEK Hussey,0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33,N,NA,MR Benson,SL Shastri,true,Win Both,"Punjab Cricket Association Stadium, Mohali",Chandigarh
27,335984,2008-04-19,MF Maharoof,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9,N,NA,Aleem Dar,GA Pratapkumar,false,Lost Toss or Match,Feroz Shah Kotla,Delhi
14,335985,2008-04-20,MV Boucher,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5,N,NA,SJ Davis,DJ Harper,false,Lost Toss or Match,Wankhede Stadium,Mumbai
15,335986,2008-04-20,DJ Hussey,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5,N,NA,BF Bowden,K Hariharan,false,Lost Toss or Match,Eden Gardens,Kolkata
20,335987,2008-04-21,SR Watson,0,Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,bat,Rajasthan Royals,wickets,6,N,NA,Aleem Dar,RB Tiffin,false,Lost Toss or Match,Sawai Mansingh Stadium,Jaipur
22,335988,2008-04-22,V Sehwag,0,Deccan Chargers,Delhi Daredevils,Deccan Chargers,bat,Delhi Daredevils,wickets,9,N,NA,IL Howell,AM Saheba,false,Lost Toss or Match,"Rajiv Gandhi International Stadium, Uppal",Hyderabad
29,335989,2008-04-23,ML Hayden,0,Chennai Super Kings,Mumbai Indians,Mumbai Indians,field,Chennai Super Kings,runs,6,N,NA,DJ Harper,GA Pratapkumar,false,Lost Toss or Match,"MA Chidambaram Stadium, Chepauk",Chennai
22,335990,2008-04-24,YK Pathan,0,Deccan Chargers,Rajasthan Royals,Rajasthan Royals,field,Rajasthan Royals,wickets,3,N,NA,Asad Rauf,MR Benson,true,Win Both,"Rajiv Gandhi International Stadium, Uppal",Hyderabad
31,335991,2008-04-25,KC Sangakkara,0,Kings XI Punjab,Mumbai Indians,Mumbai Indians,field,Kings XI Punjab,runs,66,N,NA,Aleem Dar,AM Saheba,false,Lost Toss or Match,"Punjab Cricket Association Stadium, Mohali",Chandigarh


In [0]:
import json
class Database():
    def __init__(self,loc):
        self.loc = loc
    def connect_to_db(self):
        self.connection = sqlite3.connect(self.loc)
        self.cursor = self.connection.cursor()
        print("Connection Established")
    def add_table(self,df,table_name,schema):
        # Add Dataframe as a table to Database 
        self.cursor.execute(f'''CREATE TABLE {table_name} ({schema})''')
        df = df.toPandas()
        df.to_sql(f'{table_name}', self.connection, if_exists='append', index = False)
        print(f"Loaded {table_name} into Database Successfully ")
    def drop_table(self,table_name):
        # Drop table from Database
        try:
            self.cursor.execute(f"""Drop table {table_name}""")
        except:
            print('Already Deleted')
    def execute_query(self,query):
        try :
            query_res = self.cursor.execute(f"""{query}""")
            return json.dumps(query_res.fetchall())
        except sqlite3.Error as err:
            print('Query Syntax Error')
            return None
    def get_status(self):
        try:
            self.connection.cursor()
            print('Connection Present')
        except :
            print('No Connection present')
        return None
    def close(self):
        self.connection.close()

In [0]:
venue_schema = 'venue_id int,venue text,city text'
match_schema = 'match_id int,date text,player_of_match text,venue_id int,neutral_venue int,team1 text,team2 text,toss_winner text,toss_decision text,winner text,result text,result_margin text,eliminator text, method text,umpire1 text,umpire2 text'
ball_schema = 'match_id int, inning int, overs int, ball int, batsman text, non_striker text, bowler text, batsman_runs float, extra_runs float, total_runs float, non_boundary float, is_wicket int, dismissal_kind text, player_dismissed text, fielder text, extras_type text, batting_team text, bowling_team text'

In [0]:
df_match = spark.read.csv(os.path.join(folder_loc,'ipl_matches.csv'),header=True,inferSchema=True)
df_venue = spark.read.csv(os.path.join(folder_loc,'ipl_venue.csv'),header=True,inferSchema=True)
df_ball = spark.read.csv(os.path.join(folder_loc,'ipl_ball_by_ball.csv'),header=True,inferSchema=True)

In [0]:
import sqlite3
ipl = Database('demo.db')
ipl.connect_to_db()
ipl.add_table(df_venue,'venue',venue_schema)
ipl.add_table(df_match,'match',match_schema)
ipl.add_table(df_ball,'ball',ball_schema)

Connection Established
Loaded venue into Database Successfully 
Loaded match into Database Successfully 
Loaded ball into Database Successfully 


In [0]:
ipl.execute_query("""
    select venue.venue
    from (select venue_id,count(*) as col1 
            from (select * from match where eliminator = 'Y') group by venue_id order by col1 DESC limit 3 )as t1 inner join 
    venue on t1.venue_id = venue.venue_id 

""")

Out[30]: '[["Dubai International Cricket Stadium"], ["Sheikh Zayed Stadium"], ["Sardar Patel Stadium, Motera"]]'

In [0]:
ipl.execute_query("""
    select fielder,count(*) as col1
    from (select * from ball where dismissal_kind = 'caught') group by fielder order by col1 DESC limit 1
""")

Out[31]: '[["KD Karthik", 118]]'

In [0]:
ipl.execute_query(""" 
    select bowler from ball 
    where match_id in (select match_id from match where method = 'D/L') and dismissal_kind != 'stumped' and is_wicket = 1
    group by bowler order by count(*) DESC limit 1
""")

Out[32]: '[["AB Dinda"]]'

In [0]:
ipl.execute_query("""
    select t1.batsman,100*t2.runs/t1.balls from 
    (select batsman,count(*) as balls from ball where overs > 6 and extras_type != 'noballs' group by batsman) as t1 inner join
    (select batsman,sum(batsman_runs) as runs from ball where overs > 6 group by batsman) as t2 on t1.batsman = t2.batsman
    order by 100*t2.runs/t1.balls DESC limit 1
""")

Out[33]: '[["B Stanlake", 250.0]]'

In [0]:
ipl.execute_query(""" 
    select venue.venue,venue.city,t2.total_extras from venue inner join  
    (select match.venue_id,sum(t1.extras_sum) as total_extras from match join 
    (select match_id,sum(extra_runs) as extras_sum from ball group by match_id)as t1 on match.match_id = t1.match_id 
    group by match.venue_id order by total_extras DESC limit 1) as t2 on venue.venue_id = t2.venue_id
""")

Out[34]: '[["M.Chinnaswamy Stadium", "Bengaluru", 1355.0]]'

In [0]:
ipl.execute_query("""
    select player_of_match from 
    (select player_of_match,count(*) as col1 from match where neutral_venue = 1 group by player_of_match) 
    where col1 = 
    (select max(col1) from (select player_of_match,count(*) as col1 from match 
     where neutral_venue = 1 group by player_of_match))
""")

Out[35]: '[["GJ Maxwell"], ["JH Kallis"], ["JP Duminy"], ["YK Pathan"]]'

In [0]:
ipl.execute_query("""
    select t1.batsman,t1.run_scored/t2.outs as average from 
    (select batsman,sum(batsman_runs) as run_scored from ball group by batsman) as t1 inner join 
    (select player_dismissed,count(*) as outs from ball where player_dismissed != 'NA' group by player_dismissed) as t2 on
    t1.batsman = t2.player_dismissed order by average DESC limit 10
""")

Out[36]: '[["MN van Wyk", 55.666666666666664], ["RD Gaikwad", 51.0], ["AC Voges", 45.25], ["KL Rahul", 44.86440677966102], ["HM Amla", 44.38461538461539], ["Iqbal Abdulla", 44.0], ["DA Warner", 42.71544715447155], ["JM Bairstow", 41.578947368421055], ["CH Gayle", 41.13793103448276], ["MS Dhoni", 40.991150442477874]]'

In [0]:
ipl.execute_query("""
    select umpire1,u1+u2 from 
    (select umpire1,count(umpire1) as u1 from match group by umpire1)as t1 inner join
    (select umpire2,count(umpire2) as u2 from match group by umpire2)as t2 
    on t1.umpire1 = t2.umpire2 order by u1+u2 DESC limit 1
""")

Out[37]: '[["S Ravi", 121]]'

In [0]:
ipl.execute_query("""
    select venue.venue,venue.city from venue inner join 
    (select venue_id from match inner join 
    (select match_id,sum(batsman_runs) as score from ball where batsman = 'V Kohli' group by match_id 
    order by score DESC limit 1) as t1 where match.match_id = t1.match_id) as t2 on venue.venue_id = t2.venue_id
""")

Out[38]: '[["M.Chinnaswamy Stadium", "Bengaluru"]]'

In [0]:
ipl.close()